In [3]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit
import pickle

In [32]:
df = pd.read_csv('training_data.csv')


In [33]:
df.head()

,Patient ID,Recording locations:,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,...,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Outcome,Campaign,Additional ID
0,2530,AV+PV+TV+MV,Child,Female,98.0,15.9,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
1,9979,AV+PV+TV+MV,Child,Female,103.0,13.1,False,Present,AV+MV+PV+TV,TV,...,High,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
2,9983,AV+PV+TV+MV,Child,Male,115.0,19.1,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
3,13918,AV+PV+TV+MV,Child,Male,98.0,15.9,False,Present,TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
4,14241,AV+PV+TV+MV,Child,Male,87.0,11.2,False,Present,AV+MV+PV+TV,PV,...,Low,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN


In [34]:
df['Outcome'].value_counts()

Outcome
Normal      486
Abnormal    456
Name: count, dtype: int64

In [35]:
df['Murmur'].value_counts()

Murmur
Absent     695
Present    179
Unknown     68
Name: count, dtype: int64

In [36]:
result = df.groupby('Murmur')['Patient ID'].agg(list).reset_index()
result

,Murmur,Patient ID
0,Absent,"[2530, 14998, 23625, 24160, 31737, 38337, 3904..."
1,Present,"[9979, 13918, 14241, 29045, 33151, 36327, 4084..."
2,Unknown,"[9983, 29378, 39456, 49558, 49607, 49618, 4963..."


In [37]:

# Lấy ra 3 array
array_Absent = result.loc[result['Murmur'] == 'Absent', 'Patient ID'].values[0]
array_Present = result.loc[result['Murmur'] == 'Present', 'Patient ID'].values[0]
array_Unknown = result.loc[result['Murmur'] == 'Unknown', 'Patient ID'].values[0]

print("Array cho Murmur A:", array_Absent,len(array_Absent))
print("Array cho Murmur P:", array_Present,len(array_Present))
print("Array cho Murmur U:", array_Unknown,len(array_Unknown))
print("Array cho Murmur A:", len(array_Absent))
print("Array cho Murmur P:", len(array_Present))
print("Array cho Murmur U:", len(array_Unknown))

Array cho Murmur A: [2530, 14998, 23625, 24160, 31737, 38337, 39043, 39403, 40058, 40798, 43852, 46532, 49561, 49562, 49568, 49577, 49585, 49598, 49610, 49622, 49631, 49641, 49653, 49659, 49661, 49678, 49683, 49687, 49704, 49719, 49729, 49735, 49745, 49776, 49808, 49829, 49832, 49839, 49842, 49853, 49854, 49873, 49876, 49896, 49930, 49946, 49952, 49959, 49960, 49963, 49968, 49969, 49970, 49974, 49978, 49979, 49980, 49983, 49986, 49987, 49988, 49990, 49993, 49995, 49998, 49999, 50001, 50004, 50005, 50006, 50007, 50008, 50009, 50012, 50014, 50015, 50017, 50018, 50023, 50026, 50027, 50029, 50032, 50034, 50037, 50043, 50047, 50048, 50049, 50053, 50057, 50061, 50066, 50067, 50070, 50072, 50074, 50075, 50076, 50077, 50078, 50079, 50080, 50085, 50086, 50089, 50092, 50094, 50096, 50100, 50103, 50104, 50105, 50109, 50111, 50113, 50117, 50119, 50121, 50122, 50125, 50126, 50128, 50133, 50137, 50141, 50142, 50143, 50145, 50146, 50151, 50153, 50155, 50160, 50164, 50165, 50166, 50168, 50174, 50204, 

In [38]:
result = df.groupby('Outcome')['Patient ID'].agg(list).reset_index()
result

,Outcome,Patient ID
0,Abnormal,"[2530, 9979, 9983, 13918, 14241, 14998, 23625,..."
1,Normal,"[43852, 46532, 49568, 49572, 49577, 49598, 496..."


In [39]:
array_Abnormal = result.loc[result['Outcome'] == 'Abnormal', 'Patient ID'].values[0]
array_Normal = result.loc[result['Outcome'] == 'Normal', 'Patient ID'].values[0]

print("Array cho Murmur A:", array_Absent,len(array_Abnormal))
print("Array cho Murmur N:", array_Present,len(array_Normal))
print("Array cho Murmur A:", len(array_Abnormal))
print("Array cho Murmur N:", len(array_Normal))

Array cho Murmur A: [2530, 14998, 23625, 24160, 31737, 38337, 39043, 39403, 40058, 40798, 43852, 46532, 49561, 49562, 49568, 49577, 49585, 49598, 49610, 49622, 49631, 49641, 49653, 49659, 49661, 49678, 49683, 49687, 49704, 49719, 49729, 49735, 49745, 49776, 49808, 49829, 49832, 49839, 49842, 49853, 49854, 49873, 49876, 49896, 49930, 49946, 49952, 49959, 49960, 49963, 49968, 49969, 49970, 49974, 49978, 49979, 49980, 49983, 49986, 49987, 49988, 49990, 49993, 49995, 49998, 49999, 50001, 50004, 50005, 50006, 50007, 50008, 50009, 50012, 50014, 50015, 50017, 50018, 50023, 50026, 50027, 50029, 50032, 50034, 50037, 50043, 50047, 50048, 50049, 50053, 50057, 50061, 50066, 50067, 50070, 50072, 50074, 50075, 50076, 50077, 50078, 50079, 50080, 50085, 50086, 50089, 50092, 50094, 50096, 50100, 50103, 50104, 50105, 50109, 50111, 50113, 50117, 50119, 50121, 50122, 50125, 50126, 50128, 50133, 50137, 50141, 50142, 50143, 50145, 50146, 50151, 50153, 50155, 50160, 50164, 50165, 50166, 50168, 50174, 50204, 

In [44]:
array1 = array_Abnormal
array = [array_Absent,array_Present,array_Unknown]

array_out_Abnormal = [[],[],[]]
i = 0
for array2 in array:
    array_out_Abnormal[i] = np.intersect1d(array1, array2)
    i = i+1
print(len(array_out_Abnormal[0]),len(array_out_Abnormal[1]),len(array_out_Abnormal[2]))

263 150 43


In [45]:
array1 = array_Normal
array = [array_Absent,array_Present,array_Unknown]

array_out_Normal = [[],[],[]]
i = 0
for array2 in array:
    array_out_Normal[i] = np.intersect1d(array1, array2)
    i = i+1
print(len(array_out_Normal[0]),len(array_out_Normal[1]),len(array_out_Normal[2]))

432 29 25


In [48]:
x_test =[]
for i in range(len(array_out_Abnormal)):
    if i == 0 :
        size =  70
    elif i == 1:
        size = 18
    elif i ==2:
        size =7    
    sampled_array = np.random.choice(array_out_Abnormal[i], size=size, replace=False)
    x_test.extend(sampled_array)
    print(len(x_test))

70
88
95


In [49]:
for i in range(len(array_out_Normal)):
    if i == 0 :
        size =  70
    elif i == 1:
        size = 18
    elif i ==2:
        size =7    
    sampled_array = np.random.choice(array_out_Normal[i], size=size, replace=False)
    x_test.extend(sampled_array)
    print(len(x_test))

165
183
190


In [174]:
df['Patient ID']

0       2530
1       9979
2       9983
3      13918
4      14241
       ...  
937    85340
938    85341
939    85343
940    85345
941    85349
Name: Patient ID, Length: 942, dtype: int64

In [51]:


# Sử dụng numpy.setdiff1d để loại bỏ giá trị giống nhau
id_train = np.setdiff1d(df['Patient ID'], x_test)

print("Mảng sau khi loại bỏ giá trị giống nhau:", len(id_train))

Mảng sau khi loại bỏ giá trị giống nhau: 752


In [52]:
# file = open('id_patient.csv','wb')
# pickle.dump((id_train,x_test),file)
# file.close()

In [4]:
file = open('id_patient.csv','rb')
data = pickle.load(file)
id_train,id_test = data
file.close()

In [5]:
def time_start(filename):
    file_tsv = filename[:-4]+".tsv"
    df_time = pd.read_csv(str(file_tsv))
    df_time = np.array(df_time)
    # print(len(df_time))
    # print(df[0][0].split('\t')[0])
    # print(df[len(df)-1][0].split('\t')[0])
    start = (df_time[0][0].split('\t')[0])
    end = (df_time[len(df_time)-1][0].split('\t')[0])
    return start+"_"+end


In [7]:

data = 'training_data'

out_file = 'training_data_all/'
df = pd.read_csv('training_data.csv')
for element in id_test:
    for filename in os.listdir(data):
        if filename.startswith(str(element)) and filename.endswith('.wav'):
            # print(filename)
            link_file = os.path.join(data,filename)
            # print(link_file)
            time = (time_start(link_file))

            murmur_value = df.loc[df['Patient ID'] == element, 'Murmur'].values[0]
            outcome_value = df.loc[df['Patient ID'] == element, 'Outcome'].values[0]
            # print(f"For Patient ID {element}:")
            # print(murmur_value)
            # print(outcome_value)
            # print(filename)
            
            shutil.copy(link_file, out_file +'test/'+murmur_value+'/'+outcome_value+'_'+str(time)+'_'+filename)

    

In [8]:

data = 'training_data'

out_file = 'training_data_all/'
df = pd.read_csv('training_data.csv')
for element in id_train:
    for filename in os.listdir(data):
        if filename.startswith(str(element)) and filename.endswith('.wav'):
            link_file = os.path.join(data,filename)
            # print(link_file)
            time = (time_start(link_file))
 
            # file_txt = (name_file.split('_')[0])+'_'+(name_file.split('_')[1])+'.txt'
            # # print(file_txt)
            # df = pd.read_csv(file_txt)
        # Lấy giá trị của cột "Murmur" và "Outcome" dựa trên Patient ID
            murmur_value = df.loc[df['Patient ID'] == element, 'Murmur'].values[0]
            outcome_value = df.loc[df['Patient ID'] == element, 'Outcome'].values[0]
            # print(f"For Patient ID {element}:")
            # print(murmur_value)
            # print(outcome_value)
            # print(filename)
            shutil.copy(link_file, out_file +'train/'+murmur_value+'/'+outcome_value+'_'+str(time)+'_'+filename)

    